# import 패키지

In [1]:
from konlpy.tag import Okt
from keras import models
from keras import preprocessing

import numpy as np
import pandas as pd
import re
import os
import time

In [2]:
# 경로 이동
os.chdir(r'./\ICT해커톤\챗봇\데이터')

In [3]:
# 사전 정의
PAD = "<PADDING>"
PAD_INDEX = 0
OOV = "<OOV>"
OOV_INDEX = 1
user_name='defalut값'

okt= Okt()
talk= True
index=0
data = {'bot_q':[],'user_a':[]}
bot_a = ''
bot_s = ''
bot_a_list = {'고정':['나를 상징하는 것을 저렇게 표현하신 이유가 무엇인가요?\n',
                    '왜 그런 생각이 드셨는지 설명해 주실 수 있나요?\n',
                    f'최근에 {user_name}님이 본인을 그렇게 느낄만한 일들이 있었나요?\n',
                    '그런 상황에서 스쳐지나간 생각이 있으신가요?\n',
                    '주변 사람이 이런 생각을 가지고 있다면 뭐라고 말을 해주겠어요?\n',
                    '그렇지만 본인에 대해 지금까지 알아가는 시간이 부족했던거 같아요 어떻게 생각하시나요?\n',
                    '그래도 오늘 이렇게 이야기를 나누면서 기분이 좀 풀리셨나요?, 앞으로도 제가 도와드릴께요 함께해요!'],
              '시작':['안녕하세요 엘리🐘에요! 저와 함께 얘기를 나누어 봐요!','혹시라도 중간에 멈추고 싶으시면 끝내기라고 입력해주세요!'],
         '안되다':'많이 정신 없으셨나봐요 😥',
         '공감':f'맞아요 제 생각도 그래요. {user_name}님은 충분히 노력하셨어요.',
             '종료':'\n종료합니다',
             '마무리':'그래도 오늘 이렇게 이야기를 나누면서 기분이 좀 풀리셨나요? \n앞으로도 제가 도와드릴께요 함께해요!'}
bot_a_emotion={'행복':'와 너무 행복하셨겠어요!! 기쁨은 나누면 배가 된데요!! 제가 제곱으로 늘려드릴께요!!!',
              '분노':'대따대따 화나셨겠다.. 어떻게 참으셨어요 대단해요!👍',
              '중립':'아.. 그러셨구나 충분히 그럴 수 있죠!',
              '슬픔':'마음아프시겠어요 ..😢 슬픔은 나누면 나눠진다고 했어요..!'}



# BiLSTM

In [4]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 인풋이 리스트면
    if isinstance(sentences,list):
    # 모든 문장 반복
        for sentence in sentences:
            # [\"':;~()] 특수기호 제거
            sentence = re.sub("[ㄱ-ㅎㅏ-ㅣ-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", " ", sentence)
            
            
            # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
            sentence = " ".join(tagger.morphs(sentence))
            sentences_pos.append(sentence)
            
    # str이면        
    elif isinstance(sentences, str):
        sentences=re.sub("[ㄱ-ㅎㅏ-ㅣ-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]", " ", sentences)
        sentences_pos= " ".join(tagger.morphs(sentences))
        
    return sentences_pos

In [5]:
def convert_text_to_index_for_classification(sentences, vocabulary): 
    sentences_index = []
    if isinstance(sentences,list):
        # 모든 문장에 대해서 반복
        for sentence in sentences:
            sentence_index = []

            # 문장의 단어들을 띄어쓰기로 분리
            for word in sentence.split():
                if vocabulary.get(word) is not None:
                    # 사전에 있는 단어면 해당 인덱스를 추가
                    sentence_index.extend([vocabulary[word]])
                else:
                    # 사전에 없는 단어면 OOV 인덱스를 추가
                    sentence_index.extend([vocabulary[OOV]])

            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]

            # 최대 길이에 없는 공간은 패딩 인덱스로 채움
            sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]

            # 문장의 인덱스 배열을 추가
            sentences_index.append(sentence_index)
    elif isinstance(sentences, str):
        sentence_index = []
        for word in sentences.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        if len(sentence_index) > max_sequences:
            sentence_index = sentence_index[:max_sequences]

        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]

        sentences_index.append(sentence_index)
    return sentences_index

In [6]:
# 파일 불러오기
df_main = pd.read_csv('sentences.csv',encoding='utf-8')
print(df_main.head())
a=df_main['Emotion'].unique()
category =list(a)
CATEGORY = len(category)

# 분노 0,슬픔 1, 중립 2, 행복 3,
# 카테고리 인덱스

category_to_index = {word: index for index, word in enumerate(category)}

index_to_category = {index: word for index, word in enumerate(category)}


                      Sentence Emotion
0  서해 수자원도 한류라서 그동안 불법조업 방치했나?      분노
1                     니가 인간이가.      분노
2    강아지를 때리고 유기하는것만 학대는 아닙니다!      분노
3        하여간 우리나라 교육엔 미래가 없어!!      분노
4     채널에이 군부 개인거 인증하냐 동아일보야!!      분노


In [7]:
words = []
ori_sentence =[]
# 데이터 프레임 list화
for i in range(len(df_main)):
    tmp =[]
    tmp.append(str(i+1))
    tmp.append(df_main.iloc[i].Sentence)
    tmp.append(df_main.iloc[i].Emotion)
    ori_sentence.append(tmp)
    
# 문장 형태소 분석 및 전처리
sente=[]
for i in ori_sentence:
    sente.append(i[1])

senten = pos_tag(sente)

# 단어들의 배열 생성
for sentence in senten:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, OOV]

VOCAB_SIZE = len(words)
# print(VOCAB_SIZE)

In [8]:
# 문장 길이 확인
max_length = max(len(l) for l in senten)
avg_length = sum(map(len, senten))/len(senten)
# print('리뷰의 최대 길이 : {}'.format(max_length))
# print('리뷰의 평균 길이 : {}'.format(avg_length))
max_sequences= int(avg_length) + 15

In [9]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [10]:
# 모델 불러오기
load_c_model = models.load_model('main_lstm_cl.h5')

In [11]:
def predict_emotion(text):
    pre_text = pos_tag(text)
    pre_x=convert_text_to_index_for_classification(pre_text, word_to_index)
    result=np.argmax(load_c_model.predict(np.asarray(pre_x).reshape(1,max_sequences)))
    return index_to_category[result]

# Chatbot_elly

In [20]:
def generate_Q(user_a, idx):
    print('\n - - - - - - \n')
    # 챗봇 질문
    bot_a=bot_a_list['고정'][idx]

    # 대화 내용 저장
    data['bot_q'].append(bot_a)
    data['user_a'].append(user_a)

    if idx == 2:
        # 유저 대답 감정 분석 후 맞는 질문 저장
        user_emotion = predict_emotion(user_a)
        bot_react=bot_a_emotion[user_emotion]
    elif idx == 4:
        bot_react=bot_a_list['공감']
    else :
        bot_react=''
            
        
    return bot_a, bot_react
    # 유저 질문 변수 초기화 질문 인덱스 값 증가 및 3초 지연
    

In [23]:
# 끝내기 전까지 반복
for i in range(len(bot_a_list['고정'])+1):
    if i==0:
        # 초반 멘트
        for ment in bot_a_list['시작']:
            print(ment)
            
        print('- - - -')
        print(bot_a_list['고정'][0])
        continue
    user=input()
    # 유저가 원하면 종료
    if user == '끝내기':
        print(bot_a_list['종료'])
        break
    elif i < len(bot_a_list['고정']):
        bot_q, bot_re=generate_Q(user,i)
        if bot_re != '':
            print(bot_re)
        print(bot_q)
    else :
        print(bot_a_list['마무리'])
    time.sleep(3)
    
# data.items()

안녕하세요 엘리🐘에요! 저와 함께 얘기를 나누어 봐요!
혹시라도 중간에 멈추고 싶으시면 끝내기라고 입력해주세요!
- - - -
나를 상징하는 것을 저렇게 표현하신 이유가 무엇인가요?

활화산처럼 불이 붙었었지만 지금은 그렇지 않고 꺼져버린 것 같다.

 - - - - - - 

왜 그런 생각이 드셨는지 설명해 주실 수 있나요?

지금은 뭔가 차가워졌고 쉬어야 할 것 같은 기분이라 그랬습니다.

 - - - - - - 

아.. 그러셨구나 충분히 그럴 수 있죠!
최근에 defalut값님이 본인을 그렇게 느낄만한 일들이 있었나요?

날씨가 추워져서 수족냉증이 심해져서 몸 컨디션이 안좋아 졌어요. 

 - - - - - - 

그런 상황에서 스쳐지나간 생각이 있으신가요?

따뜻한 곳에 가서 쉬고싶다.

 - - - - - - 

맞아요 제 생각도 그래요. defalut값님은 충분히 노력하셨어요.
주변 사람이 이런 생각을 가지고 있다면 뭐라고 말을 해주겠어요?

어서 들어가 쉬어.. 쉬면서 해야지..

 - - - - - - 

그렇지만 본인에 대해 지금까지 알아가는 시간이 부족했던거 같아요 어떻게 생각하시나요?

맞아요...

 - - - - - - 

그래도 오늘 이렇게 이야기를 나누면서 기분이 좀 풀리셨나요?, 앞으로도 제가 도와드릴께요 함께해요!
네 고마워요
그래도 오늘 이렇게 이야기를 나누면서 기분이 좀 풀리셨나요? 
앞으로도 제가 도와드릴께요 함께해요!
